# EID 444 Homework 2

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

#[1] https://stackoverflow.com/questions/29241056/how-do-i-use-np-newaxis

#R is a matrix of particle positions for N L-J particles



def displacements(R):
    '''Calculates all displacements between all pairs 
    of particles'''

    #Computes are pairwise displacements: r_i - r_j
    #[1] - Learned about np.newaxis to help nduct matrix operations withour consuming much memory
    dR = R[:, np.newaxis, :] - R[np.newaxis, :, :]

    return dR



R = np.array([[1, 2], [3, 4]])  # Two particles

dR = displacements(R)

print(dR)







[[[ 0  0]
  [-2 -2]]

 [[ 2  2]
  [ 0  0]]]


# LJ Forces Function

In [ ]:
def lj_forces(dR, sigma, epsilon):

     rij = np.linalg.norm(dR, axis=-1)
     
     rij_inv = np.where(rij > 0,  )

     term_1 = -1 * r_unit_vec * (4 * epsilon / (sigma))
     term_2 = -12 * ((sigma / rij)**13)
     term_3 = 6 * ((sigma / rij)**7)

     FRR = term_1 * (term_2 + term_3)

     return FRR


# Net Forces


In [24]:
def net_forces(FRR):

    FR = np.sum(FRR, axis=1)
    return FR

# Verlet step

In [25]:
def verlet_step(r_current, r_prev, t, m , FR):
    ''' Calculate the verlet step of the pendulum-spring 
    system'''

    acceleration = FR / m 

    r_next = (2*r_current) - r_prev + ((acceleration) * (t**2))

    return r_next

# Simulation

In [26]:
def simulation(R_0, v_0, m, t, t_max, sigma, epsilon):

    n_steps = int(t_max / t)
    trajectory = np.empty((n_steps,2))

    r_prev = R_0
    r_current = r_prev + (v_0 * t)

    trajectory[0] = r_prev
    trajectory[1] = r_current

    for step in range(2, n_steps):
        
        r_next = verlet_step(trajectory[step - 1], trajectory[step - 2], t, m, FR)
        trajectory[step] = r_next
        r_previous = trajectory[step - 1]
    
    return trajectory

# Plot and Animate Trajectory

In [27]:
def plot_traj(trajectory):
    ''' Plots the trajectory on a grid'''

    plt.figure()
    plt.plot(trajectory[:, 0], trajectory[:, 1], label = 'Trajectory')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Simulated Trajectory')
    plt.axis('equal')
    plt.legend()
    plt.show()


def animate_traj(trajectory):
    '''Animates trajectory and saves it as a GIF'''

    fig,ax = plt.subplots()
    ax.set_xlim(np.min(trajectory[:, 0]) - 1, np.max(trajectory[:, 0]) + 1)
    ax.set_ylim(np.min(trajectory[:, 1]) - 1, np.max(trajectory[:, 1]) + 1)
    ax.set_aspect('equal')

    point, = ax.plot([], [], 'bo')  # The mass
    spring, = ax.plot([], [], 'r-', lw=2)  # The spring

    def update(frame):
        point.set_data(trajectory[frame, 0], trajectory[frame, 1])
        spring.set_data([0, trajectory[frame, 0]], [0, trajectory[frame, 1]])
        return point, spring

    ani = animation.FuncAnimation(fig, update, frames=range(0, len(trajectory), 10), blit=True, interval=50)
    ani.save("trajectory.gif", writer="pillow")
    plt.show()











# 2 Particles where forces are zero

In [28]:
sigma = 1.0
epsilon = 1.0
r_min = (2**(1/6)) * sigma

R1 = np.array([[0, 0], [r_min,0]]) 
dR1 = displacements(R1)
FRR1 = lj_forces(dR1, sigma, epsilon)
FR1 = net_forces(FRR1)

print(f' Net force of Case 1: {FR1}')

R2 = np.array([[0,0], [-r_min, r_min]])
dR2 = displacements(R2)
FRR2 = lj_forces(dR2, sigma, epsilon)
FR2 = net_forces(FRR2)

print(f'Net force of Case 2: {FR2}')



ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()